In [31]:
import numpy as np
import pandas as pd
import math

In [32]:
a = pd.read_csv("iris.csv")
data = a.drop(["sepal_width","petal_length"],axis =1)
df = data.sample(frac=1)          #shuffle
r,c = data.shape
# reset index values

xx  =df.reset_index()

df = xx.iloc[:,1:4]
kf=df.iloc[106:150,:]
df = df.iloc[0:105,:]


In [33]:
#this function returns mean and varianc of the features
# df = data frame with header and last column as output/classification

#note : input only one type of label/class

def basics(df):   
    #data  =df.values
    r,c = df.shape          #get sample size and features count
    x = df.iloc[:,0:c-1]    #features
    y = df.iloc[:,c-1]
    #x = df.values
    mean = []
    var = []
    for i in range(c-1):      # calculate mean for input features
        m = np.mean(x.values[:,i])
        mean.append(m)
        
    for i in range(c-1):   # calculate variance of each features
        v = np.var(x.values[:,i])
        var.append(v)
        
    return mean,var 

In [34]:
#this function pandas dataframe as input (have features and labels)
#returns 

def class_separator(df):
    r,c = df.shape
    dt1 =[]
    dt2 = []
    dt3 =[]
    y_cls = df.iloc[:,c-1].unique()            #get all classifier/different output
    l = len(y_cls)    # count of different label
    for i in range(r):  #iterate over each sample/row
        if df.iloc[i,c-1] == y_cls[0]:
            dt  = df.iloc[i,0:c]
            dt1.append(dt)
            
        elif df.iloc[i,c-1] == y_cls[1]:
            dtm = df.iloc[i,0:c]
            dt2.append(dtm)
            
        elif df.iloc[i,c-1] == y_cls[2]:
            dti = df.iloc[i,0:c]
            dt3.append(dti)
            
            
    return pd.DataFrame(dt1),pd.DataFrame(dt2),pd.DataFrame(dt3)
                
                
 

In [35]:
#mean = 1*2 array having mean values of each features
#var = 1*2 array having variance of each features
# x = [x1,x2]     features values to be predicted
def basic_prob(mean,var,x):
    l = len(mean)
    p = 1
    for i in range(l):
        a = 2*(math.pi)*(np.square(var[i])) 
        b = ((x[i] - mean[i])**2)/(2*np.square(var[i]))
        c = np.exp(-1*b)
        p = p * ((a**(-0.5))*c)       #gaussian
        
    return p

In [36]:
# x = [sepal_length,petal_width]
#df = [sepal_lengh,petal_width,label]

def main(df,x):
    r,c = df.shape
    y_cls = df.iloc[:,c-1].unique()  
    y_cls[0],y_cls[1],y_cls[2] = class_separator(df)   # get separate data for each classifier
    r,c = df.shape
    r1,c1 = y_cls[0].shape
    r2,c2 = y_cls[1].shape
    r3,c3 = y_cls[2].shape
    
    # calculate the probability of each classifier in training set
    
    
    #calculate mean and variance of each labels
    
    mean1,var1  = basics(y_cls[0])
    mean2,var2  = basics(y_cls[1])
    mean3,var3 = basics(y_cls[2])
    
    # calculate probailities for each lables w.r.to given points (only numerator, as denominator remains same for all)
    
    # calculate prob. of setosa
    
    prob1 = (r1/r)*basic_prob(mean1,var1,x)
    prob2 = (r2/r)*basic_prob(mean2,var2,x)
    prob3 = (r3/r)*basic_prob(mean3,var3,x)
    
    return prob1,prob2,prob3
    
    

In [37]:
def prediction(df,x):
    r,c = df.shape
    #print("The given features represents:")
    prob = main(df,x)    # get the probabilities values
    ind = np.argmax(prob)
    label =df.iloc[:,c-1].unique()
    out = label[ind]
    
    return out

In [38]:
def prediction_matrix(train_df,pool_df):
    r,c = pool_df.shape
    pre = []
    for i in range(r):
        m = prediction(train_df,pool_df.iloc[i,0:2])
        pre.append(m)
        
    return pre

In [39]:
#df = data used for prediction (with label)
#pre = predicted data (only label)

#pre = prediction_matrix(df,df)


def accuracy(actual,pre):
    r = len(pre)           #total cases/count for prediction
    #actual_label = actual.iloc[:,-1]
    count = 0
    
    for i in range(r):
        if actual.iloc[i,-1] == pre[i]:
            count = count + 1
            
    acc = (count/r)*100
            
    return acc
    




In [40]:
pre = prediction_matrix(df,kf)
accuracy(kf,pre)


90.9090909090909

In [11]:
#Q1.b
import matplotlib.pyplot  as plt

In [12]:
#data splitting into test,train and pool


r,c = df.shape
a =round(r*0.1)
b = round(r*0.6)    # count of pool set samples


train_df1 = df.iloc[0:a,:]       #10%
pool_df1 = df.iloc[a:a+b+1,:]    #60%
test_df1 = df.iloc[a+b+1:,:]     #30%


xx  =df.reset_index()

df = xx.iloc[:,1:4]


# resetting the index of train,pool and test dataframes
xx1 = train_df1.reset_index()
train_df = xx1.iloc[:,1:4]


xx2 = pool_df1.reset_index()
pool_df = xx2.iloc[:,1:4]


xx3 = test_df1.reset_index()
test_df = xx3.iloc[:,1:4]


In [13]:
def uncertainity_index(train_df,pool_df):
    r,c = pool_df.shape
    prob1 = []
    prob2 = []
    prob3 = []
    for i in range(r):
        p,q,r = main(train_df,pool_df.iloc[i,0:2])
        prob1.append(p)
        prob2.append(q)
        prob3.append(r)
        
    t1 = np.argmin(prob1)
    t2 = np.argmin(prob2)
    t3 = np.argmin(prob3)
    
    ind = min(np.array([t1,t2,t3]))
    
    return ind

In [14]:
print("The uncertainity index is :")
print(uncertainity_index(train_df,pool_df))
print("pool data :")
print(pool_df.iloc[0,:])
print("prediction:")
print(prediction(df,np.array([5.8,1.2])))

The uncertainity index is :
0
pool data :
sepal_length           6.4
petal_width            1.3
species         versicolor
Name: 0, dtype: object
prediction:
versicolor


In [15]:
ind = uncertainity_index(train_df,pool_df)
ind

0

In [23]:
def active_learn(train_df,pool_df):
    itr = 10       # itertions
    acc = []      # stores accurancy % for each iterations
    for i in range(itr):
        pre = prediction_matrix(train_df,pool_df)      # predictions
        acp = accuracy(pool_df,pre)                    #accuracy %
        acc.append(acp)
        
        index  = uncertainity_index(train_df,pool_df)     #get the uncertainity index
        #print(pool_df)
        tt = train_df.append(pool_df.iloc[index,:])    #update train set
        
        #reset index
        xx1 = tt.reset_index()
        train_df = xx1.iloc[:,1:4]
        
        
        pp = pool_df.drop([index])        #drop uncertainity row from pool data frame
        #reset index
        xx2 = pp.reset_index()
        pool_df = xx2.iloc[:,1:4]
        
        
    return acc

In [24]:
import time


acc =active_learn(train_df,pool_df)



acc

[67.1875,
 61.904761904761905,
 61.29032258064516,
 63.934426229508205,
 83.33333333333334,
 84.7457627118644,
 89.65517241379311,
 96.49122807017544,
 94.64285714285714,
 90.9090909090909]

# b.iii

In [30]:
import random
acc1 = []

for j in range(10):
    r,c = pool_df.shape
    i = random.randint(0,r-1)

    pool_df.drop(i)
    train_df.append(pool_df.iloc[i,:]) 
    
    pre = prediction_matrix(train_df,pool_df)
    acc1.append( accuracy(pool_df,pre)
    
acc1


[92.0185,
 62.8015,
 72.9795,
 60.9105,
 65.3565,
 71.6895,
 74.6605,
 87.9075,
 61.966499999999996,
 85.7555]